In [1]:
# Imports
!pip install --upgrade tensorflow
!pip install tensorflow-gpu
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install datasets
!pip install tokenizers==0.7.0
!pip install transformers==2.8.0
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu
!pip install wget
!pip install openpyxl
!pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 394.3 MB 13 kB/s s eta 0:00:01     |█                               | 11.5 MB 3.9 MB/s eta 0:01:39 |█                               | 13.4 MB 3.9 MB/s eta 0:01:3915.8 MB 3.9 MB/s eta 0:01:3818.5 MB 3.9 MB/s eta 0:01:3809 eta 0:00:08 25.0 MB 46.6 MB/s eta 0:00:08| 27.1 MB 46.6 MB/s eta 0:00:08��                             | 30.9 MB 46.6 MB/s eta 0:00:086.6 MB/s eta 0:00:080:08s eta 0:00:083 MB 46.6 MB/s eta 0:00:08                   | 43.1 MB 46.6 MB/s eta 0:00:08 MB 46.6 MB/s eta 0:00:08 MB 46.6 MB/s eta 0:00:08�███                            | 49.6 MB 46.6 MB/s eta 0:00:08██▏                           | 51.9 MB 46.6 MB/s eta 0:00:08eta 0:00:08�                           | 56.1 MB 46.6 MB/s eta 0:00:08 |████▉                           | 59.0 MB 46.6 MB/s eta 0:00:08    | 61.2 MB 46.6 MB/s eta 0:00:08     |█████▏                          | 63.6 MB 44.3 MB/s eta 0:00:08e

In [40]:
# Imports
import datasets
import pandas as pd
import csv

from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import EncoderDecoderModel, BertTokenizer, BertTokenizerFast, DistilBertTokenizer, DistilBertTokenizerFast

In [21]:
# Test CUDA
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print("Device:", device)

Device: cuda


In [22]:
# Load swiss data
path_corpus = "data_train.csv"

data_txt = []
data_ref = []

with open("data_train.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_ALL)
    next(reader, None)

    for row in reader:
        data_txt.append(row[0])
        data_ref.append(row[1])

tuples = list(zip(data_txt, data_ref))
dataframe = pd.DataFrame(tuples, columns=["article", "highlights"])

In [25]:
# Load german data
tuples = pd.read_excel("data_train_test.xlsx", engine="openpyxl")
del tuples["Unnamed: 0"]
dataframe = pd.concat([dataframe, tuples])
dataframe = dataframe.dropna()

In [26]:
# Concat swiss and german data
german_data = datasets.arrow_dataset.Dataset.from_pandas(dataframe[["article", "highlights"]])
german_data = german_data.shuffle()
print(dataframe)

                                                 article  \
0      Minghella war der Sohn italienisch-schottische...   
1      Ende der 1940er Jahre wurde eine erste Auteur-...   
2      Al Pacino, geboren in Manhattan, ist der Sohn ...   
3      Der Name der Alkalimetalle leitet sich von dem...   
4      Die Arbeit ist bereits seit dem Altertum Gegen...   
...                                                  ...   
12533  Das in der Kritik stehende Kommando Spezialkrä...   
12534  Franka Lu ist eine chinesische Journalistin un...   
12535  Nach drei Angriffen mit explosiven Postsendung...   
12536  Russische Behörden haben nach eigenen Angaben ...   
12537  DIE ZEIT: Herr Heisterhagen, Ralf Stegner kand...   

                                              highlights  
0      Anthony Minghella, CBE war ein britischer Film...  
1      Die Auteur-Theorie ist eine Filmtheorie und di...  
2      Alfredo James "Al" Pacino ist ein US-amerikani...  
3      Als Alkalimetalle werden die chemisc

In [27]:
# Split data
train_data = german_data.select(range(0, 85000))
val_data = german_data.select(range(85000, 95000))
test_data = german_data.select(range(95000, 100000))

In [28]:
# Load english data
'''
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:5%]")
'''

'\ntrain_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")\nval_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")\ntest_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:5%]")\n'

In [29]:
# Explore corpus
df = pd.DataFrame(train_data)

text_list = []
summary_list = []

for index, row in df.iterrows():
    text = row["article"]
    summary = row["highlights"]
    text_list.append(len(text))
    summary_list.append(len(summary))
    
print(sum(text_list) / len(text_list))
print(sum(summary_list) / len(summary_list))

3915.0795294117647
234.61944705882354


In [41]:
# Explore corpus
train_data.info.description
df = pd.DataFrame(train_data[:1])

# del df["id"]

for column, typ in train_data.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])

display(HTML(df.to_html()))

,article,highlights,__index_level_0__
0,"Nach dem Besuch der Lateinschule und dem bestandenen Landexamen durchlief Paul Schanz den Bildungsweg der Priesteramtskandidaten Württembergs. So besuchte er von 1857 bis 1861 das Königliche Gymnasium und das Niedere Konvikt in Rottweil und wurde nach seiner Maturitätsprüfung in das Höhere Konvikt, das Wilhelmsstift in Tübingen, aufgenommen und war dort Mitglied in der Theologengesellschaft Herzynia. 1861–65 studierte er an der württembergischen Landesuniversität Tübingen Philologie, Philosophie und Katholische Theologie. Dieser Ausbildung schloss sich das einjährige Priesterseminar in Rottenburg am Neckar an. Nach der Priesterweihe im Jahr 1866 wurde Schanz Vikar in Schramberg. Während dieser Zeit wurde er von der Philosophischen Fakultät der Universität Tübingen mit einer religionsgeschichtlichen Arbeit promoviert. 1867 wurde er Repetent am Wilhelmsstift. Neben der Dogmatik war er hier für die Repetition der naturwissenschaftlichen Fächer zuständig. 1869 unterzog er sich dem realistischen Professoratsexamen. Ein Jahr später trat er eine wissenschaftliche Reise an, die ihn nach Paris und Berlin führte. In Berlin studierte Schanz an der Friedrich-Wilhelms-Universität. Kurz nach seiner Rückkehr wurde er zum Gymnasialprofessor für Mathematik und Naturwissenschaften am Rottweiler Gymnasium ernannt. Obwohl Schanz als Naturwissenschaftler galt, wurde er 1876 zum Nachfolger von Moritz von Aberle und damit zum Universitätsprofessor für Neues Testament in Tübingen ernannt. Dies lag nicht nur daran, dass er seit etlichen Jahren der beste Absolvent der Fakultät war, sondern auch an dem Vertrauen der Fakultät ""in die Eignung der eigenen Absolventen und die Bekanntheit mit ihren nicht nur fachlichen Fähigkeiten, sondern ebenso mit ihren kirchenpolitischen Anschauungen"". Mit der Ernennung zum Professor erhielt Schanz von der Fakultät den Dr. theol. verliehen. Im Abstand von jeweils zwei Jahren entstanden nun seine vier Evangelienkommentare, die nach dem Urteil des Neutestamentlers Josef Schmid ""die bedeutendste exegetische Leistung in der katholischen Theologie des neunzehnten Jahrhunderts"" seien. 1883 wechselte Schanz innerhalb der Fakultät die Lehraufträge. Als Nachfolger von Johannes von Kuhn erhielt er die Professur für Dogmatik, der auf seinem eigenen Wunsch hin der Lehrauftrag für Apologetik hinzugefügt wurde. Das bedeutendste Werk aus dieser Zeit ist die dreibändige ""Apologie des Christentums"", durch die Schanz auch international bekannt wurde. Sein erstaunliches Wissen in den Bereichen der Philologie, der Geschichtswissenschaften, Naturwissenschaften, Exegese und Dogmatik machte es verständlich, dass er als Apologet die drängenden wissenschaftlichen Fragen seiner Zeit behandelte, vor allem die Vereinbarkeit von Evolutionstheorie und Schöpfungslehre und von Religionsgeschichte und Glaubenslehre. In den letzten Lebensjahren bekannte sich Schanz zu einem wissenschaftlichen Historismus, der in der Beschäftigung mit der Geschichte der Wissenschaft die Wissenschaft selbst entdeckte. Obwohl Schanz als eher konservativ galt, wandte er sich dennoch affirmativ den Arbeiten von evangelischen Theologen und katholischen Reformern seiner Zeit zu. So rezipierte er neben Joseph Müller, Albert Ehrhard und Alfred Loisy auch den katholischen Religionsphilosophen Maurice Blondel, dessen Werk er erstmals deutschen Lesern bekannt machte. Im Jahr 1900 wurde Paul Schanz mit dem Ehrenkreuz des Ordens der Württembergischen Krone ausgezeichnet, welches mit dem persönlichen Adelstitel verbunden war.","Paul von Schanz war ein katholischer Theologe, Professor für Mathematik, Naturwissenschaften, Neutestamentliche Exegese, Dogmatik und Apologetik.",88259


In [31]:
# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

In [32]:
# Prepare data
encoder_max_length = 512
decoder_max_length = 128
batch_size = 4 # 16

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [42]:
# Training data
train_data = train_data.shuffle()

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights"] # "id"
)

train_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

AttributeError: 'tokenizers.Encoding' object has no attribute 'n_sequences'

In [ ]:
# Validation data
val_data = val_data.shuffle()

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["article", "highlights"] # id
)

val_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

val_data = val_data.select(range(2000))

In [ ]:
# Load models
tf2tf = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-multilingual-cased", "bert-base-multilingual-cased", tie_encoder_decoder=False)
tf2tf.save_pretrained("bert2bert_multilingual")
tf2tf = EncoderDecoderModel.from_pretrained("bert2bert_multilingual")

In [ ]:
# Configure models
tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
tf2tf.config.eos_token_id = tokenizer.sep_token_id
tf2tf.config.pad_token_id = tokenizer.pad_token_id
tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size

In [ ]:
# Configure beam search
tf2tf.config.max_length = 142
tf2tf.config.min_length = 56
tf2tf.config.no_repeat_ngram_size = 3
tf2tf.config.early_stopping = True
tf2tf.config.length_penalty = 2.0
tf2tf.config.num_beams = 4

In [ ]:
# Prepare metric
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# Load checkpoint
from google.colab import drive

path_drive = "/content/drive"
drive.mount(path_drive)

path_output = path_drive + "/My Drive/Temp/Models"
path_checkpoint = path_output + "/checkpoint-16000"

tf2tf = EncoderDecoderModel.from_pretrained(path_checkpoint)
tf2tf.to("cuda")

In [ ]:
# Setup arguments
from seq2seq_training_args import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir=path_output, # "./"
    warmup_steps=1000,
    save_steps=2000,
    logging_steps=100,
    eval_steps=2000,
    save_total_limit=1,
    fp16=True
)

In [ ]:
# Start training
from seq2seq_trainer import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=tf2tf,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()

In [ ]:
# Evaluate training
def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred_summary"] = output_str

    return batch

results = test_data.map(
    generate_summary,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article"]
)

print(results[0]["pred_summary"])
print(results[0]["highlights"])
print("====================")

rouge.compute(predictions=results["pred_summary"], references=results["highlights"], rouge_types=["rouge2"])["rouge2"].mid

# tf2tf = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail").to("cuda")
# tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")